In [25]:
import xmltodict
import pandas as pd
import copy
import csv
import os
import langid
import multiprocessing
import re
from IPython.display import clear_output
BASE_DIR = '/home/antonio/lattes_base/Collection'
N_THREADS = 3

In [26]:
def get_info(doc, dout):
    author = pd.DataFrame(columns=['author_id', 'author_name', 'author_ref', 'doctorate', 'paper_issn', 'paper_ano', 'paper', 'paper_home_page'])
    
    
    ### get articles by authors without id
    try:
        author_id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
    except:
        author_id = None

    ### get author info
    try:
        artigos = doc['CURRICULO-VITAE']['PRODUCAO-BIBLIOGRAFICA']['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']
        author_name = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-EM-CITACOES-BIBLIOGRAFICAS']
        author_complete_name = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-COMPLETO']
    except:
        return author
    
    list_artigos = []
    papers = {}
    
    ###get articles
    if type(artigos) == list: ### if the author has several articles
        for artigo in artigos:   
            try:
                if(artigo['DADOS-BASICOS-DO-ARTIGO']['@IDIOMA'] == "Inglês"):
                    papers['paper'] = artigo['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO']
                    papers['paper_issn'] = artigo['DETALHAMENTO-DO-ARTIGO']['@ISSN']
                    papers['paper_ano'] = artigo['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO']
                    papers['paper_home_page'] = artigo['DADOS-BASICOS-DO-ARTIGO']['@HOME-PAGE-DO-TRABALHO']       
                    string = ''
                    for autores in artigo['AUTORES']: ##get co-authors
                        try:
                            string += 'ordem: ' + str(autores['@ORDEM-DE-AUTORIA']) + '\n'
                            string += 'name: ' + autores['@NOME-COMPLETO-DO-AUTOR'] + '\n'
                            string += 'name_ref ' + autores['@NOME-PARA-CITACAO']  + '\n'
                        except:
                            pass
                    papers['coauthors'] = string
                    list_artigos.append(copy.deepcopy(papers))
            except:
                pass
    else: ## or only one articles
        if(artigos['DADOS-BASICOS-DO-ARTIGO']['@IDIOMA'] == "Inglês"):
                    papers['paper'] = artigos['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO']
                    papers['paper_issn'] = artigos['DETALHAMENTO-DO-ARTIGO']['@ISSN']
                    papers['paper_ano'] = artigos['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO']
                    papers['paper_home_page'] = artigos['DADOS-BASICOS-DO-ARTIGO']['@HOME-PAGE-DO-TRABALHO']       
                    string = ''
                    for autores in artigos['AUTORES']: ##get co-authors
                        try:
                            string += 'ordem: ' + str(autores['@ORDEM-DE-AUTORIA']) + '\n'
                            string += 'name: ' + autores['@NOME-COMPLETO-DO-AUTOR'] + '\n'
                            string += 'name_ref ' + autores['@NOME-PARA-CITACAO']  + '\n'
                        except:
                            pass
                    papers['coauthors'] = string
                    list_artigos.append(copy.deepcopy(papers))

    ### fills the dataframe
    for index in range(len(list_artigos)):
        author.loc[index] = (author_id, author_complete_name, author_name, dout, list_artigos[index]['paper_issn'], list_artigos[index]['paper_ano'], list_artigos[index]['paper'], list_artigos[index]['paper_home_page'])
    return author

In [27]:
def get_author(doc, courses):
    author = pd.DataFrame(columns=['author_id', 'author_name', 'author_ref', 'doctorate', 'paper_issn', 'paper_ano', 'paper', 'paper_home_page'])
    
    ### checks if the author has a doctorate
    try :
        dout  = doc['CURRICULO-VITAE']['DADOS-GERAIS']['FORMACAO-ACADEMICA-TITULACAO']['DOUTORADO']
    except:
        dout = None
    
    if dout:
        if type(dout) == list: ### if the author has several doctorates
            l_dout = [x['@NOME-CURSO'] for x in dout]
            for value in l_dout:
                if value and value in courses:
                    dout = ", ".join(l_dout)
                    author = get_info(doc, dout)
                    break
                
        elif dout['@NOME-CURSO'] and dout['@NOME-CURSO'] in courses: ### or only one doctorate
            dout = dout['@NOME-CURSO'] 
            author = get_info(doc, dout)
            
    return author

In [28]:
def worker(files, return_num, return_dict, courses):
    author = pd.DataFrame(columns=['author_id', 'author_name', 'author_ref', 'doctorate', 'paper_issn', 'paper_ano', 'paper', 'paper_home_page'])
    dataframes = []
    dataframes.append(author.copy())
    
    n_files = len(files)
    for index, file in enumerate(files, start=0):
        with open(file + '/curriculo.xml', encoding="ISO-8859-1") as fd:
            doc = xmltodict.parse(fd.read())
        new_df = get_author(doc, courses)
        if not new_df.empty:
            dataframes.append(new_df)
        clear_output(wait=True)
        print(str("%.2f" % (index * 100 / n_files /4)) + '%')
        
    author = pd.concat(dataframes)
    return_dict[return_num] = author

### get all files

In [29]:
files_list = []

for folder in os.listdir(BASE_DIR):
    path = os.path.join(BASE_DIR, folder)
    if(os.path.isdir(path) == True and not folder.startswith('.')):
        for files in os.listdir(path):
            if not files.startswith('.'):
                file = os.path.join(path, files)
                files_list.append(file)

### Define tasks

In [ ]:
file = open('courses.txt', 'r')
courses = file.read()
file.close()
courses = re.split(', |\n',courses)

n_threads = N_THREADS
n_data = len(files_list)
if(n_data%n_threads == 0):
        division = int(n_data/n_threads)
else:
    division = int(n_data/n_threads)+1
    
manager = multiprocessing.Manager()
return_dict = manager.dict()
jobs = []
for i in range(n_threads):
    init = i*division
    end = (i+1)*division
    p = multiprocessing.Process(target=worker, args=(files_list[init:end], i, return_dict, courses))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

### Generate dataframe and save

In [ ]:
author = pd.DataFrame(columns=['author_id', 'author_name', 'author_ref', 'doctorate', 'paper_issn', 'paper_ano', 'paper', 'paper_home_page'])
dataframes = []
dataframes.append(author.copy())
for value in return_dict.values():
    dataframes.append(value)
author = pd.concat(dataframes)

In [ ]:
author

In [ ]:
author.to_csv('data.csv', sep='|')

In [ ]:
'''author = {}
author_inf = {}

author_name = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-EM-CITACOES-BIBLIOGRAFICAS']
author_id = doc['CURRICULO-VITAE']['@NUMERO-IDENTIFICADOR']
author_complete_name = doc['CURRICULO-VITAE']['DADOS-GERAIS']['@NOME-COMPLETO']

author_inf['id'] = author_id
author_inf['name'] = author_complete_name
author_inf['ref_name'] = author_name
author['informacoes'] = author_inf

artigos = doc['CURRICULO-VITAE']['PRODUCAO-BIBLIOGRAFICA']['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']
pub = []
for artigo in artigos:
    if(artigo['DADOS-BASICOS-DO-ARTIGO']['@IDIOMA'] == "Inglês"):
        paper = {}
        paper['titulo'] = artigo['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO']
        paper['issn'] = artigo['DETALHAMENTO-DO-ARTIGO']['@ISSN']
        paper['ano'] = artigo['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO']
        paper['home_page'] = artigo['DADOS-BASICOS-DO-ARTIGO']['@HOME-PAGE-DO-TRABALHO']       
        
        
        #print(artigo['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO'])
        #print(artigo['DETALHAMENTO-DO-ARTIGO']['@ISSN'])
        #print(artigo['DADOS-BASICOS-DO-ARTIGO']['@HOME-PAGE-DO-TRABALHO'])
        #print(artigo['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO'])
        #authors = []
        #authors_inf = {}
        #for autores in artigo['AUTORES']:            
        #    authors_inf['nome'] = autores['@NOME-COMPLETO-DO-AUTOR']
        #    authors_inf['name_ref']  = autores['@NOME-PARA-CITACAO']
        #    authors_inf['ordem'] = autores['@ORDEM-DE-AUTORIA']
            #authors[autores['@NOME-PARA-CITACAO']] = autors_inf.copy()
        #    authors.append(copy.deepcopy(authors_inf))
            
            #print(autores['@NOME-COMPLETO-DO-AUTOR'])
            #print(autores['@NOME-PARA-CITACAO'])
            #print(autores['@ORDEM-DE-AUTORIA'])
            
        #paper['autores'] = copy.deepcopy(authors)
        pub.append(copy.deepcopy(paper))
        print('\n\n')
author['artigos'] = pub[:]'''